# Previous settings

In [1]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import joblib
import pandas as pd
import os 

# Suppress the warning caused by sklearn
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Setting the route to read the data set
PROJECT_ROOT_DIR = "."
DATA_PATH = os.path.join(PROJECT_ROOT_DIR, "datasets")
os.makedirs(DATA_PATH, exist_ok=True)

# Setting the route to save the result of prediction
RESULT_PATH = os.path.join(PROJECT_ROOT_DIR, "results")
os.makedirs(RESULT_PATH, exist_ok=True)

# Setting the route to load the model
MODEL_PATH = os.path.join(PROJECT_ROOT_DIR, "models")
os.makedirs(MODEL_PATH, exist_ok=True)

# Data reading and preprocessing

In [3]:
# Read the excel of the data
data = os.path.join(DATA_PATH,"dataset2.xlsx")
df1 = pd.read_excel(data)
df1

,SiO2,TiO2,Al2O3,FeO*,MgO,CaO,Na2O,K2O,Rb,Sr,...,La,Ce,Nd,Sm,Eu,Yb,Hf,Pb,Th,U
0,50.7849,1.26603,14.7489,8.85043,8.57204,12.4174,2.69378,0.070098,1.020000,123.000000,...,2.520000,7.910000,8.020000,2.810000,1.040000,2.710000,1.990000,0.332000,0.132000,0.033000
1,51.2255,1.35886,15.1206,8.78775,8.03930,11.6706,2.87478,0.170470,4.440000,133.000000,...,3.550000,10.450000,9.330000,3.060000,1.100000,2.900000,2.380000,0.767000,0.497000,0.131000
2,50.7967,1.41358,14.8134,9.40325,8.72019,11.4585,2.78884,0.046481,1.110000,88.000000,...,1.890000,7.180000,8.570000,3.150000,1.120000,3.120000,2.280000,0.315000,0.117000,0.023000
3,50.8165,1.22237,15.4801,8.24114,8.17716,11.6460,2.77904,0.445845,12.340000,255.000000,...,7.880000,18.880000,12.560000,3.350000,1.170000,2.140000,2.210000,1.313000,1.441000,0.379000
4,50.8683,1.17347,15.4212,8.69922,8.77606,11.9609,2.38612,0.089985,1.350000,103.000000,...,2.660000,8.290000,8.000000,2.740000,0.940000,2.830000,2.040000,0.348000,0.175000,0.049000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1926,48.0896,1.23878,17.2892,9.28234,8.85247,11.8391,2.68249,0.109549,1.021629,165.733497,...,3.193728,9.207351,7.706894,2.582949,1.078329,2.309117,1.827773,0.411722,0.196951,0.065747
1927,50.9323,1.69100,15.2455,9.86600,6.94400,10.9220,2.92400,0.124000,1.097640,136.995000,...,3.717280,11.889300,10.887000,3.645760,1.336670,3.318880,2.765340,0.533924,0.159052,0.069925
1928,51.4630,1.36600,15.8350,8.37900,7.49130,11.1810,2.83300,0.290000,6.161750,175.192000,...,6.641170,16.744600,10.773600,3.136230,1.206130,2.461500,2.145910,0.687221,0.609537,0.210778
1929,50.9203,1.11180,14.2816,11.22910,8.00494,11.6536,2.19327,0.042449,0.910000,70.083000,...,1.456000,4.904000,5.691000,2.323000,0.879000,3.078000,1.648000,0.382000,0.076000,0.027000


In [4]:
# Add element ratios
df1['La/Sm'] = df1['La']/df1['Sm']
df1['Ce/Pb'] = df1['Ce']/df1['Pb']
df1['Ba/Nb'] = df1['Ba']/df1['Nb']
df1['Sm/Yb'] = df1['Sm']/df1['Yb']
df1

,SiO2,TiO2,Al2O3,FeO*,MgO,CaO,Na2O,K2O,Rb,Sr,...,Eu,Yb,Hf,Pb,Th,U,La/Sm,Ce/Pb,Ba/Nb,Sm/Yb
0,50.7849,1.26603,14.7489,8.85043,8.57204,12.4174,2.69378,0.070098,1.020000,123.000000,...,1.040000,2.710000,1.990000,0.332000,0.132000,0.033000,0.896797,23.825301,8.181818,1.036900
1,51.2255,1.35886,15.1206,8.78775,8.03930,11.6706,2.87478,0.170470,4.440000,133.000000,...,1.100000,2.900000,2.380000,0.767000,0.497000,0.131000,1.160131,13.624511,12.086957,1.055172
2,50.7967,1.41358,14.8134,9.40325,8.72019,11.4585,2.78884,0.046481,1.110000,88.000000,...,1.120000,3.120000,2.280000,0.315000,0.117000,0.023000,0.600000,22.793651,10.916667,1.009615
3,50.8165,1.22237,15.4801,8.24114,8.17716,11.6460,2.77904,0.445845,12.340000,255.000000,...,1.170000,2.140000,2.210000,1.313000,1.441000,0.379000,2.352239,14.379284,28.055077,1.565421
4,50.8683,1.17347,15.4212,8.69922,8.77606,11.9609,2.38612,0.089985,1.350000,103.000000,...,0.940000,2.830000,2.040000,0.348000,0.175000,0.049000,0.970803,23.821839,8.285714,0.968198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1926,48.0896,1.23878,17.2892,9.28234,8.85247,11.8391,2.68249,0.109549,1.021629,165.733497,...,1.078329,2.309117,1.827773,0.411722,0.196951,0.065747,1.236466,22.363040,5.885862,1.118587
1927,50.9323,1.69100,15.2455,9.86600,6.94400,10.9220,2.92400,0.124000,1.097640,136.995000,...,1.336670,3.318880,2.765340,0.533924,0.159052,0.069925,1.019617,22.267776,3.581479,1.098491
1928,51.4630,1.36600,15.8350,8.37900,7.49130,11.1810,2.83300,0.290000,6.161750,175.192000,...,1.206130,2.461500,2.145910,0.687221,0.609537,0.210778,2.117565,24.365670,6.522988,1.274113
1929,50.9203,1.11180,14.2816,11.22910,8.00494,11.6536,2.19327,0.042449,0.910000,70.083000,...,0.879000,3.078000,1.648000,0.382000,0.076000,0.027000,0.626776,12.837696,7.620910,0.754711


In [5]:
X_all = df1
X_all

,SiO2,TiO2,Al2O3,FeO*,MgO,CaO,Na2O,K2O,Rb,Sr,...,Eu,Yb,Hf,Pb,Th,U,La/Sm,Ce/Pb,Ba/Nb,Sm/Yb
0,50.7849,1.26603,14.7489,8.85043,8.57204,12.4174,2.69378,0.070098,1.020000,123.000000,...,1.040000,2.710000,1.990000,0.332000,0.132000,0.033000,0.896797,23.825301,8.181818,1.036900
1,51.2255,1.35886,15.1206,8.78775,8.03930,11.6706,2.87478,0.170470,4.440000,133.000000,...,1.100000,2.900000,2.380000,0.767000,0.497000,0.131000,1.160131,13.624511,12.086957,1.055172
2,50.7967,1.41358,14.8134,9.40325,8.72019,11.4585,2.78884,0.046481,1.110000,88.000000,...,1.120000,3.120000,2.280000,0.315000,0.117000,0.023000,0.600000,22.793651,10.916667,1.009615
3,50.8165,1.22237,15.4801,8.24114,8.17716,11.6460,2.77904,0.445845,12.340000,255.000000,...,1.170000,2.140000,2.210000,1.313000,1.441000,0.379000,2.352239,14.379284,28.055077,1.565421
4,50.8683,1.17347,15.4212,8.69922,8.77606,11.9609,2.38612,0.089985,1.350000,103.000000,...,0.940000,2.830000,2.040000,0.348000,0.175000,0.049000,0.970803,23.821839,8.285714,0.968198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1926,48.0896,1.23878,17.2892,9.28234,8.85247,11.8391,2.68249,0.109549,1.021629,165.733497,...,1.078329,2.309117,1.827773,0.411722,0.196951,0.065747,1.236466,22.363040,5.885862,1.118587
1927,50.9323,1.69100,15.2455,9.86600,6.94400,10.9220,2.92400,0.124000,1.097640,136.995000,...,1.336670,3.318880,2.765340,0.533924,0.159052,0.069925,1.019617,22.267776,3.581479,1.098491
1928,51.4630,1.36600,15.8350,8.37900,7.49130,11.1810,2.83300,0.290000,6.161750,175.192000,...,1.206130,2.461500,2.145910,0.687221,0.609537,0.210778,2.117565,24.365670,6.522988,1.274113
1929,50.9203,1.11180,14.2816,11.22910,8.00494,11.6536,2.19327,0.042449,0.910000,70.083000,...,0.879000,3.078000,1.648000,0.382000,0.076000,0.027000,0.626776,12.837696,7.620910,0.754711


# Water predicting

In [6]:
# load the trained model "Established_RFR_model.pkl"
model = os.path.join(MODEL_PATH,"Established_RFR_model.pkl")
model_rf = joblib.load(model)

In [7]:
result_rf = model_rf.predict(X_all)
np.array(result_rf,dtype=float)
result_rf

array([0.18068297, 0.26112021, 0.22481048, ..., 0.39104501, 0.20650176,
       0.20973995])

In [8]:
# Insert the predicted H2O contents into the input dataset
df1.insert(df1.shape[1], 'H2O_P', result_rf)
df1

,SiO2,TiO2,Al2O3,FeO*,MgO,CaO,Na2O,K2O,Rb,Sr,...,Yb,Hf,Pb,Th,U,La/Sm,Ce/Pb,Ba/Nb,Sm/Yb,H2O_P
0,50.7849,1.26603,14.7489,8.85043,8.57204,12.4174,2.69378,0.070098,1.020000,123.000000,...,2.710000,1.990000,0.332000,0.132000,0.033000,0.896797,23.825301,8.181818,1.036900,0.180683
1,51.2255,1.35886,15.1206,8.78775,8.03930,11.6706,2.87478,0.170470,4.440000,133.000000,...,2.900000,2.380000,0.767000,0.497000,0.131000,1.160131,13.624511,12.086957,1.055172,0.261120
2,50.7967,1.41358,14.8134,9.40325,8.72019,11.4585,2.78884,0.046481,1.110000,88.000000,...,3.120000,2.280000,0.315000,0.117000,0.023000,0.600000,22.793651,10.916667,1.009615,0.224810
3,50.8165,1.22237,15.4801,8.24114,8.17716,11.6460,2.77904,0.445845,12.340000,255.000000,...,2.140000,2.210000,1.313000,1.441000,0.379000,2.352239,14.379284,28.055077,1.565421,0.521484
4,50.8683,1.17347,15.4212,8.69922,8.77606,11.9609,2.38612,0.089985,1.350000,103.000000,...,2.830000,2.040000,0.348000,0.175000,0.049000,0.970803,23.821839,8.285714,0.968198,0.194801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1926,48.0896,1.23878,17.2892,9.28234,8.85247,11.8391,2.68249,0.109549,1.021629,165.733497,...,2.309117,1.827773,0.411722,0.196951,0.065747,1.236466,22.363040,5.885862,1.118587,0.218570
1927,50.9323,1.69100,15.2455,9.86600,6.94400,10.9220,2.92400,0.124000,1.097640,136.995000,...,3.318880,2.765340,0.533924,0.159052,0.069925,1.019617,22.267776,3.581479,1.098491,0.261965
1928,51.4630,1.36600,15.8350,8.37900,7.49130,11.1810,2.83300,0.290000,6.161750,175.192000,...,2.461500,2.145910,0.687221,0.609537,0.210778,2.117565,24.365670,6.522988,1.274113,0.391045
1929,50.9203,1.11180,14.2816,11.22910,8.00494,11.6536,2.19327,0.042449,0.910000,70.083000,...,3.078000,1.648000,0.382000,0.076000,0.027000,0.626776,12.837696,7.620910,0.754711,0.206502


In [9]:
# Output results
result = os.path.join(RESULT_PATH,"result.xlsx")
df1.to_excel(result,"result.xlsx")